In [26]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from package import encoder

from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error, r2_score, mean_absolute_percentage_error
from scipy.stats import pearsonr

import seaborn as sns; sns.set()

NomeBase = "LamostMedium"

In [27]:
from keras.layers import Input, Dense, LeakyReLU
from sklearn.model_selection import train_test_split
from keras.callbacks import TensorBoard
from keras import regularizers

import math
import keras
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from keras.models import Model

from pympler import muppy, summary

In [28]:
def print_memory_summary():
    all_objects = muppy.get_objects()
    sum1 = summary.summarize(all_objects)
    summary.print_(sum1)

## Modelo para treinamento

In [29]:
def training_model():
    input_x0 = Input(shape=(66,), name='inp0')
    encoded0 = Dense(224, name='encoded0',kernel_regularizer=regularizers.l2(0.00005),kernel_initializer = "HeUniform")(input_x0)
    a0=LeakyReLU(alpha=0.1)(encoded0)
    encoded1 = Dense(320, name='encoded1',kernel_regularizer=regularizers.l2(0.00005),kernel_initializer = "HeUniform")(a0)
    a1=LeakyReLU(alpha=0.1)(encoded1)
    encoded2 = Dense(608, name='encoded2',kernel_regularizer=regularizers.l2(0.00005),kernel_initializer = "HeUniform")(a1)
    a2=LeakyReLU(alpha=0.1)(encoded2)
    encoded3 = Dense(320, name='encoded3',kernel_regularizer=regularizers.l2(0.00005),kernel_initializer = "HeUniform")(a2)
    a3=LeakyReLU(alpha=0.1)(encoded3)
    encoded4 = Dense(128, name='encoded4',kernel_regularizer=regularizers.l2(0.00005),kernel_initializer = "HeUniform")(a3)
    a4=LeakyReLU(alpha=0.1)(encoded4)
    encoded5 = Dense(64, name='encoded5',kernel_regularizer=regularizers.l2(0.00005),kernel_initializer = "HeUniform")(a4)
    a5=LeakyReLU(alpha=0.1)(encoded5)
    output_y0 = Dense(1,name='oup0',kernel_initializer = "HeUniform", activation='linear')(a5)
    trainable_model=Model(inputs=input_x0, outputs=output_y0)
    trainable_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001), loss='mse')
    return trainable_model

In [83]:
def saveScores(parameter, BaseSaveModel, AouG, batchSize, epochs, score, Y_test, Y_pred):
    with open(f"scores/{BaseSaveModel}/{parameter}Scores-{AouG}.txt", "a") as myfile:
        myfile.write(f"Modelo: {parameter}-{AouG}-{BaseSaveModel}\n")
        myfile.write(f"batchSize: {batchSize}\n")
        myfile.write(f"epochs: {epochs}\n")
        myfile.write(f"Accuracy: {score*100}%\n")
        myfile.write(f"MSE: {mean_squared_error(Y_test, Y_pred)}\n")
        myfile.write(f"MAE: {mean_absolute_error(Y_test, Y_pred)}\n")
        myfile.write(f"MedAE: {median_absolute_error(Y_test, Y_pred)}\n")
        myfile.write(f"MAPE: {mean_absolute_percentage_error(Y_test, Y_pred) * 100}%\n")
        myfile.write(f"---------------------------\n\n")

In [84]:
def lossPlot(history, parameter, BaseSaveModel, AouG):
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss for ' + parameter)
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper right')
    plt.savefig(f'scores/{BaseSaveModel}/loss/{parameter}-{AouG}.png', dpi=300)
    plt.show()

In [85]:
def printScores(parameter, BaseSaveModel, AouG, batchSize, epochs, score, Y_test, Y_pred):
    print(f"Modelo: {parameter}-{AouG}-{BaseSaveModel}\n")
    print(f"batchSize: {batchSize}\n")
    print(f"epochs: {epochs}\n")
    print(f"Accuracy: {score*100}%\n")
    print(f"MSE: {mean_squared_error(Y_test, Y_pred)}\n")
    print(f"MAE: {mean_absolute_error(Y_test, Y_pred)}\n")
    print(f"MedAE: {median_absolute_error(Y_test, Y_pred)}\n")
    print(f"MAPE: {mean_absolute_percentage_error(Y_test, Y_pred) * 100}%\n")
    print(f"---------------------------\n\n")

In [86]:
def train_model(parameter, BaseSaveModel, AouG, batchSizes, Epochs, X_train, Y_train, X_val, Y_val, X_test, Y_test):
    bestacc = 0
    for batchSize in batchSizes:
        for epochs in Epochs:
            if batchSize < 64 and epochs > 300:
                continue
            model = training_model()
            history = model.fit(X_train, Y_train, epochs = epochs, validation_data=(X_val, Y_val), batch_size = batchSize, verbose = 2, callbacks=[TensorBoard(log_dir='./mytensorboard')], shuffle=True, use_multiprocessing=True)
            pred = model.predict(X_test)
            score = r2_score(Y_test, pred)
            if score > bestacc:
                bestacc = score
                bestpred = pred
                bestmodel = model
                besthistory = history
                saveScores(parameter, BaseSaveModel, AouG, batchSize, epochs, score, Y_test, pred)
    bestmodel.save(f'modelosPosRio\\{BaseSaveModel}\\{parameter}-{AouG}.h5')
    print("Modelo salvo com acurácia: ", score)
    lossPlot(besthistory, parameter, BaseSaveModel, AouG)
    printScores(parameter, BaseSaveModel, AouG, batchSize, epochs, bestacc, Y_test, bestpred)
    return bestpred

# LamostMedium

## Dwarfs

In [34]:
AouG = "Dwarfs"
NomeBase = "LamostMedium"

In [35]:
AnasCores = pd.read_csv("E:\\IC\\BasesTreino\\LamostMedium-Colors-Dwarfs.csv", index_col = 'ID')

In [36]:
AnasCores

,u-J378,u-J395,u-J410,u-J430,u-g,u-J515,u-r,u-J660,u-i,u-J861,...,teff_cnn,logg_cnn,feh_cnn,alpha_m_cnn,c_fe,ca_fe,n_fe,mg_fe,si_fe,ni_fe
ID,,,,,,,,,,,,,,,,,,,,,
iDR4_3_SPLUS-n02n18_0040731,0.6183,1.0508,1.2710,1.2807,1.2519,1.3407,1.2224,1.0877,1.1028,0.9948,...,6614.94,4.099,-0.274,0.076,-0.26760,-0.03872,-0.13689,0.12154,0.08599,-0.07160
iDR4_3_SPLUS-n02n18_0034154,0.2075,0.3849,0.9449,1.0038,1.1963,1.4454,1.6982,1.7883,1.8254,1.8025,...,5645.36,4.312,-0.334,0.085,0.07561,0.08418,0.06322,0.16942,0.04026,0.04051
iDR4_3_SPLUS-n02n18_0036979,0.2639,0.5195,1.2247,1.3780,1.9246,2.1031,2.5980,2.6773,2.8487,2.9154,...,5044.25,3.626,-0.136,0.061,0.05919,0.08039,0.03626,0.13601,0.02489,0.04424
iDR4_3_SPLUS-n02n18_0011190,0.3266,0.4263,1.0195,1.0906,1.3868,1.5834,1.6034,1.7468,1.6009,1.9268,...,5632.16,4.148,0.001,0.001,-0.09541,0.01560,0.25512,-0.00759,-0.00156,0.03042
iDR4_3_SPLUS-n02n18_0013484,0.2196,0.4738,1.1639,1.3388,1.8705,2.0127,2.6149,2.7070,2.8212,2.9471,...,4776.82,4.266,-0.273,-0.021,-0.10056,0.17008,-0.13612,-0.12723,-0.14240,-0.04826
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
iDR4_3_STRIPE82-0164_0045592,0.4731,0.4658,0.8994,0.9522,1.1874,1.3375,0.3479,1.5321,0.8582,1.6254,...,6078.74,4.159,-0.228,-9999.000,-9999.00000,-9999.00000,-9999.00000,-9999.00000,-9999.00000,-9999.00000
iDR4_3_STRIPE82-0164_0032054,0.2347,0.3997,1.1143,1.1965,1.7398,1.8666,1.8041,2.5429,2.3426,2.6659,...,4953.50,4.713,-0.598,-9999.000,-9999.00000,-9999.00000,-9999.00000,-9999.00000,-9999.00000,-9999.00000
iDR4_3_STRIPE82-0164_0041441,0.2886,0.5299,1.2053,1.3902,2.0450,2.1581,2.8287,3.0373,2.9388,3.4185,...,4599.52,4.443,-0.296,-9999.000,-9999.00000,-9999.00000,-9999.00000,-9999.00000,-9999.00000,-9999.00000


In [37]:
xtrain = AnasCores.iloc[:, :66]

In [38]:
xtrain

,u-J378,u-J395,u-J410,u-J430,u-g,u-J515,u-r,u-J660,u-i,u-J861,...,r-J660,r-i,r-J861,r-z,J660-i,J660-J861,J660-z,i-J861,i-z,J861-z
ID,,,,,,,,,,,,,,,,,,,,,
iDR4_3_SPLUS-n02n18_0040731,0.6183,1.0508,1.2710,1.2807,1.2519,1.3407,1.2224,1.0877,1.1028,0.9948,...,-0.1347,-0.1196,-0.2276,-0.2137,0.0151,-0.0929,-0.0790,-0.1080,-0.0941,0.0139
iDR4_3_SPLUS-n02n18_0034154,0.2075,0.3849,0.9449,1.0038,1.1963,1.4454,1.6982,1.7883,1.8254,1.8025,...,0.0901,0.1272,0.1043,0.1038,0.0371,0.0142,0.0137,-0.0229,-0.0234,-0.0005
iDR4_3_SPLUS-n02n18_0036979,0.2639,0.5195,1.2247,1.3780,1.9246,2.1031,2.5980,2.6773,2.8487,2.9154,...,0.0793,0.2507,0.3174,0.3562,0.1714,0.2381,0.2769,0.0667,0.1055,0.0388
iDR4_3_SPLUS-n02n18_0011190,0.3266,0.4263,1.0195,1.0906,1.3868,1.5834,1.6034,1.7468,1.6009,1.9268,...,0.1434,-0.0025,0.3234,0.1141,-0.1459,0.1800,-0.0293,0.3259,0.1166,-0.2093
iDR4_3_SPLUS-n02n18_0013484,0.2196,0.4738,1.1639,1.3388,1.8705,2.0127,2.6149,2.7070,2.8212,2.9471,...,0.0921,0.2063,0.3322,0.3433,0.1142,0.2401,0.2512,0.1259,0.1370,0.0111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
iDR4_3_STRIPE82-0164_0045592,0.4731,0.4658,0.8994,0.9522,1.1874,1.3375,0.3479,1.5321,0.8582,1.6254,...,1.1842,0.5103,1.2775,1.2295,-0.6739,0.0933,0.0453,0.7672,0.7192,-0.0480
iDR4_3_STRIPE82-0164_0032054,0.2347,0.3997,1.1143,1.1965,1.7398,1.8666,1.8041,2.5429,2.3426,2.6659,...,0.7388,0.5385,0.8618,0.8282,-0.2003,0.1230,0.0894,0.3233,0.2897,-0.0336
iDR4_3_STRIPE82-0164_0041441,0.2886,0.5299,1.2053,1.3902,2.0450,2.1581,2.8287,3.0373,2.9388,3.4185,...,0.2086,0.1101,0.5898,0.5577,-0.0985,0.3812,0.3491,0.4797,0.4476,-0.0321


In [39]:
columnYtrain =  ['teff_cnn', 'logg_cnn', 'feh_cnn', 'alpha_m_cnn', 
                 'c_fe', 'ca_fe', 'n_fe',
                 'mg_fe', 'si_fe', 'ni_fe']

In [40]:
ytrain = AnasCores[columnYtrain]

In [41]:
ytrain['teffEncoded'] = encoder.encode_DataFrame(pd.DataFrame(ytrain['teff_cnn']), 'teffMedium')

C:\Users\alber\AppData\Local\Temp\ipykernel_29200\3238081216.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ytrain['teffEncoded'] = encoder.encode_DataFrame(pd.DataFrame(ytrain['teff_cnn']), 'teffMedium')


In [42]:
#encoder.save_encoders()

In [43]:
train_x= encoder.encode_Colors(xtrain, "Dwarfs") #The input vector: 66 stellar colors

train_y = ytrain

c:\Users\alber\Envs\analiseIC\MachineLearningSplus\RedeNeural\package\encoder.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = encoders[nameC].fit_transform(pd.DataFrame(df[column]))
c:\Users\alber\Envs\analiseIC\MachineLearningSplus\RedeNeural\package\encoder.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = encoders[nameC].fit_transform(pd.DataFrame(df[column]))
c:\Users\alber\Envs\analiseIC\MachineLearningSplus\RedeNeural\package\encoder.py:58: SettingWithCopyWarning: 
A 

##### Separando base para Treinamento da temperatura, logg e feh

In [44]:
# Separando a base para treino e para teste
X_train, X_test, Y_train, Y_test = train_test_split(train_x, train_y, test_size=0.2, random_state=11085)

In [45]:
# Separando a base de treino para utilizar como validação no treinamento
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.1, random_state=11085)

In [46]:
X_train.shape

(2665, 66)

In [47]:
Y_train

,teff_cnn,logg_cnn,feh_cnn,alpha_m_cnn,c_fe,ca_fe,n_fe,mg_fe,si_fe,ni_fe,teffEncoded
ID,,,,,,,,,,,
iDR4_3_SPLUS-s02s26_0005116,5885.54,4.266,-0.140,0.020,-0.00651,0.07106,0.28749,0.07225,0.00488,-0.00420,0.361428
iDR4_3_STRIPE82-0030_0024134,5847.62,3.860,-0.489,0.126,0.14767,0.21414,0.04253,0.22676,0.08979,-0.06067,0.336390
iDR4_3_STRIPE82-0052_0026120,5304.36,4.542,-0.091,-0.012,-0.03709,0.01314,0.04713,0.04450,-0.02452,-0.00457,-0.022311
iDR4_3_STRIPE82-0132_0042427,5805.73,4.268,-0.686,0.189,0.19569,0.18189,-0.11842,0.26081,0.17328,-0.00317,0.308731
iDR4_3_STRIPE82-0132_0008211,5517.52,4.448,-0.078,-0.018,-0.08193,0.00242,0.01999,-0.03045,-0.02595,-0.02534,0.118434
...,...,...,...,...,...,...,...,...,...,...,...
iDR4_3_STRIPE82-0004_0010044,5408.61,4.372,-0.772,0.214,0.06898,0.28655,-0.20496,0.29743,0.16761,0.03707,0.046523
iDR4_3_STRIPE82-0048_0020497,5798.10,4.292,-0.021,-0.012,-0.04513,-0.01666,0.17366,-0.00542,-0.00932,0.01954,0.303694
iDR4_3_STRIPE82-0101_0038087,5657.67,3.692,-0.298,-9999.000,-9999.00000,-9999.00000,-9999.00000,-9999.00000,-9999.00000,-9999.00000,0.210971


### Teff | Logg | FEH

In [57]:
Parametros = ["TEFF", "LOGG", "FEH"]
columnTrain = ['teffEncoded', 'logg_cnn', 'feh_cnn']

AouG = "Dwarfs"
NomeBase = "LamostMedium"
BaseSaveModel = "Lamost-MR"

In [58]:
batchSizes = [32, 64, 128, 256]
Epochs = [100, 200, 300, 400, 500]

In [ ]:
predicts = []
for i in range(0, 3, 1):
    predicts.append(train_model(Parametros[i], BaseSaveModel, AouG, batchSizes, Epochs, X_train, Y_train[columnTrain[i]], X_val, Y_val[columnTrain[i]], X_test, Y_test[columnTrain[i]]))

In [60]:
predTeff = predicts[0]
predLogg = predicts[1]
predFeh = predicts[2]

In [63]:
predTeffaux = encoder.decode_DataFrame(predTeff, 'teffMedium')
Y_test['TEFF_PredictDecoded'] = predTeffaux
Y_test['LOGG_Predict'] = predLogg
Y_test['FEH_Predict'] = predFeh

In [64]:
Y_test

,teff_cnn,logg_cnn,feh_cnn,alpha_m_cnn,c_fe,ca_fe,n_fe,mg_fe,si_fe,ni_fe,teffEncoded,TEFF_PredictDecoded,LOGG_Predict,FEH_Predict
ID,,,,,,,,,,,,,,
iDR4_3_STRIPE82-0005_0028605,5685.09,4.253,-0.497,0.165,0.17985,0.08121,0.06669,0.24823,0.16309,0.05935,0.229076,5551.028320,4.244528,-0.667896
iDR4_3_STRIPE82-0048_0013477,5253.51,4.347,-0.302,0.159,0.04655,0.17966,-0.01043,0.21871,0.10999,0.04033,-0.055886,5373.220703,4.455763,-0.347771
iDR4_3_STRIPE82-0030_0047340,5725.44,4.436,-0.209,-0.005,-0.00348,-0.02077,0.15071,0.00338,0.00995,0.00871,0.255718,5746.011230,4.211554,-0.038095
iDR4_3_STRIPE82-0130_0008941,5013.25,4.198,-0.038,-0.005,-0.07105,0.05844,0.06584,-0.04190,-0.04802,0.01047,-0.214523,5006.041016,4.545473,-0.005748
iDR4_3_STRIPE82-0006_0012431,5116.29,4.289,0.058,-0.032,-0.00450,-0.01086,0.09835,-0.01338,-0.02291,0.03095,-0.146489,5106.911133,4.485288,-0.109736
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
iDR4_3_SPLUS-n02n28_0020503,5228.77,3.948,-0.136,0.089,0.07635,0.09427,-0.04551,0.16613,0.05183,0.06135,-0.072221,5266.686523,4.055098,-0.080389
iDR4_3_STRIPE82-0132_0019055,5652.10,4.056,-0.086,0.135,0.10474,0.10234,0.09820,0.21770,0.13217,0.03695,0.207293,5269.144043,4.337927,0.001812
iDR4_3_STRIPE82-0006_0024027,5391.94,4.308,0.159,-0.037,-0.01274,-0.05130,0.11948,-0.01593,0.04669,0.05998,0.035516,5403.247559,4.346299,0.069971


In [65]:
Y_test.to_csv(f"E:\\IC\\PredictsTreinos\\{NomeBase}\\{AouG}-TLFPredict.csv")

### Outros Parametros

In [66]:
AnasCores = pd.read_csv("E:\\IC\\BasesTreino\\LamostMedium-Colors-Dwarfs.csv", index_col = 'ID')

In [67]:
AnasCores

,u-J378,u-J395,u-J410,u-J430,u-g,u-J515,u-r,u-J660,u-i,u-J861,...,teff_cnn,logg_cnn,feh_cnn,alpha_m_cnn,c_fe,ca_fe,n_fe,mg_fe,si_fe,ni_fe
ID,,,,,,,,,,,,,,,,,,,,,
iDR4_3_SPLUS-n02n18_0040731,0.6183,1.0508,1.2710,1.2807,1.2519,1.3407,1.2224,1.0877,1.1028,0.9948,...,6614.94,4.099,-0.274,0.076,-0.26760,-0.03872,-0.13689,0.12154,0.08599,-0.07160
iDR4_3_SPLUS-n02n18_0034154,0.2075,0.3849,0.9449,1.0038,1.1963,1.4454,1.6982,1.7883,1.8254,1.8025,...,5645.36,4.312,-0.334,0.085,0.07561,0.08418,0.06322,0.16942,0.04026,0.04051
iDR4_3_SPLUS-n02n18_0036979,0.2639,0.5195,1.2247,1.3780,1.9246,2.1031,2.5980,2.6773,2.8487,2.9154,...,5044.25,3.626,-0.136,0.061,0.05919,0.08039,0.03626,0.13601,0.02489,0.04424
iDR4_3_SPLUS-n02n18_0011190,0.3266,0.4263,1.0195,1.0906,1.3868,1.5834,1.6034,1.7468,1.6009,1.9268,...,5632.16,4.148,0.001,0.001,-0.09541,0.01560,0.25512,-0.00759,-0.00156,0.03042
iDR4_3_SPLUS-n02n18_0013484,0.2196,0.4738,1.1639,1.3388,1.8705,2.0127,2.6149,2.7070,2.8212,2.9471,...,4776.82,4.266,-0.273,-0.021,-0.10056,0.17008,-0.13612,-0.12723,-0.14240,-0.04826
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
iDR4_3_STRIPE82-0164_0045592,0.4731,0.4658,0.8994,0.9522,1.1874,1.3375,0.3479,1.5321,0.8582,1.6254,...,6078.74,4.159,-0.228,-9999.000,-9999.00000,-9999.00000,-9999.00000,-9999.00000,-9999.00000,-9999.00000
iDR4_3_STRIPE82-0164_0032054,0.2347,0.3997,1.1143,1.1965,1.7398,1.8666,1.8041,2.5429,2.3426,2.6659,...,4953.50,4.713,-0.598,-9999.000,-9999.00000,-9999.00000,-9999.00000,-9999.00000,-9999.00000,-9999.00000
iDR4_3_STRIPE82-0164_0041441,0.2886,0.5299,1.2053,1.3902,2.0450,2.1581,2.8287,3.0373,2.9388,3.4185,...,4599.52,4.443,-0.296,-9999.000,-9999.00000,-9999.00000,-9999.00000,-9999.00000,-9999.00000,-9999.00000


In [68]:
print('cfe =', AnasCores['c_fe'].min())
print('cafe = ', AnasCores['ca_fe'].min())
print('nfe = ', AnasCores['n_fe'].min())
print('mgfe = ', AnasCores['mg_fe'].min())
print('sife = ', AnasCores['si_fe'].min())
print('nife = ', AnasCores['ni_fe'].min())

cfe = -9999.0
cafe =  -9999.0
nfe =  -9999.0
mgfe =  -9999.0
sife =  -9999.0
nife =  -9999.0


In [69]:
AnasCoresParams = AnasCores.loc[AnasCores['alpha_m_cnn'] != -9999]

In [70]:
print('cfe =', AnasCoresParams['c_fe'].min())
print('cafe = ', AnasCoresParams['ca_fe'].min())
print('nfe = ', AnasCoresParams['n_fe'].min())
print('mgfe = ', AnasCoresParams['mg_fe'].min())
print('sife = ', AnasCoresParams['si_fe'].min())
print('nife = ', AnasCoresParams['ni_fe'].min())

cfe = -0.28632
cafe =  -0.45868
nfe =  -0.35071
mgfe =  -0.22189
sife =  -0.17489
nife =  -0.2868


In [71]:
AnasCoresParams

,u-J378,u-J395,u-J410,u-J430,u-g,u-J515,u-r,u-J660,u-i,u-J861,...,teff_cnn,logg_cnn,feh_cnn,alpha_m_cnn,c_fe,ca_fe,n_fe,mg_fe,si_fe,ni_fe
ID,,,,,,,,,,,,,,,,,,,,,
iDR4_3_SPLUS-n02n18_0040731,0.6183,1.0508,1.2710,1.2807,1.2519,1.3407,1.2224,1.0877,1.1028,0.9948,...,6614.94,4.099,-0.274,0.076,-0.26760,-0.03872,-0.13689,0.12154,0.08599,-0.07160
iDR4_3_SPLUS-n02n18_0034154,0.2075,0.3849,0.9449,1.0038,1.1963,1.4454,1.6982,1.7883,1.8254,1.8025,...,5645.36,4.312,-0.334,0.085,0.07561,0.08418,0.06322,0.16942,0.04026,0.04051
iDR4_3_SPLUS-n02n18_0036979,0.2639,0.5195,1.2247,1.3780,1.9246,2.1031,2.5980,2.6773,2.8487,2.9154,...,5044.25,3.626,-0.136,0.061,0.05919,0.08039,0.03626,0.13601,0.02489,0.04424
iDR4_3_SPLUS-n02n18_0011190,0.3266,0.4263,1.0195,1.0906,1.3868,1.5834,1.6034,1.7468,1.6009,1.9268,...,5632.16,4.148,0.001,0.001,-0.09541,0.01560,0.25512,-0.00759,-0.00156,0.03042
iDR4_3_SPLUS-n02n18_0013484,0.2196,0.4738,1.1639,1.3388,1.8705,2.0127,2.6149,2.7070,2.8212,2.9471,...,4776.82,4.266,-0.273,-0.021,-0.10056,0.17008,-0.13612,-0.12723,-0.14240,-0.04826
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
iDR4_3_STRIPE82-0163_0040861,0.3866,0.4251,0.9477,1.0056,1.3554,1.5387,1.8074,1.8416,1.9482,1.9664,...,5416.55,3.887,-0.216,0.098,0.06780,0.07029,-0.02753,0.15527,0.07589,0.04897
iDR4_3_STRIPE82-0163_0044016,0.3823,0.4274,0.7984,0.8649,1.1274,1.2835,1.5048,1.5035,1.6009,1.5972,...,6036.54,4.244,-0.221,0.060,0.11298,-0.01196,0.10891,0.13146,0.05064,0.03619
iDR4_3_STRIPE82-0163_0039518,0.2209,0.5521,1.3819,1.6429,2.3021,2.4366,3.1314,3.2471,3.4291,3.5128,...,4923.39,4.554,0.140,-0.043,-0.02498,-0.01239,0.02498,-0.01143,-0.06438,0.01396


In [72]:
xtrain = AnasCoresParams.iloc[:, :66]

In [73]:
columnYtrain =  ['teff_cnn', 'logg_cnn', 'feh_cnn', 'alpha_m_cnn', 
                 'c_fe', 'ca_fe', 'n_fe',
                 'mg_fe', 'si_fe', 'ni_fe']

In [74]:
ytrain = AnasCoresParams[columnYtrain]

In [75]:
train_x= encoder.encode_DataFrame(xtrain, "NewCores") #The input vector: 66 stellar colors

train_y = ytrain

In [76]:
# Separando a base para treino e para teste
X_train, X_test, Y_train, Y_test = train_test_split(train_x, train_y, test_size=0.2, random_state=11085)

In [77]:
# Separando a base de treino para utilizar como validação no treinamento
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.1, random_state=11085)

In [78]:
Y_train

,teff_cnn,logg_cnn,feh_cnn,alpha_m_cnn,c_fe,ca_fe,n_fe,mg_fe,si_fe,ni_fe
ID,,,,,,,,,,
iDR4_3_SPLUS-n01s27_0019440,5748.81,4.265,-0.516,0.088,0.08189,0.08448,0.15760,0.13011,0.06925,0.02921
iDR4_3_STRIPE82-0006_0000813,5656.79,4.383,-0.177,0.022,0.02918,0.01176,0.05855,0.06574,0.02949,0.00432
iDR4_3_STRIPE82-0006_0026432,5526.96,4.278,-0.167,0.018,-0.02215,0.04075,0.06477,0.06317,0.00242,0.01880
iDR4_3_SPLUS-n02n28_0012662,5367.61,4.362,-0.385,0.014,-0.09885,0.11427,-0.01474,-0.00829,-0.02983,0.03659
iDR4_3_SPLUS-n01s27_0040044,5397.81,4.547,-0.321,0.017,0.08644,0.07661,-0.13799,0.04477,-0.00489,-0.02994
...,...,...,...,...,...,...,...,...,...,...
iDR4_3_STRIPE82-0005_0007897,5233.55,4.579,-0.343,0.031,-0.07211,0.02426,-0.09471,0.06796,0.01010,-0.02851
iDR4_3_STRIPE82-0007_0035292,5811.55,4.322,-0.554,0.179,0.14994,0.23801,-0.15317,0.27396,0.16425,0.05887
iDR4_3_STRIPE82-0005_0028525,5741.27,4.217,-0.266,0.035,0.03640,0.01903,-0.02002,0.07230,0.03535,0.02227


In [90]:
Parametros = ["Alpha", "C", "CA", "N", "MG", "SI", "NI"]
columnTrain = ['alpha_m_cnn', 'c_fe', 'ca_fe', 'n_fe', 'mg_fe', 'si_fe', 'ni_fe']

AouG = "Dwarfs"
NomeBase = "LamostMedium"
BaseSaveModel = "Lamost-MR"

In [91]:
batchSizes = [16, 32, 64, 128, 256]
Epochs = [100, 200, 300, 400, 500]

In [ ]:
predicts = []
for i in range(0, len(columnTrain), 1):
    predicts.append(train_model(Parametros[i], BaseSaveModel, AouG, batchSizes, Epochs, X_train, Y_train[columnTrain[i]], X_val, Y_val[columnTrain[i]], X_test, Y_test[columnTrain[i]]))

In [95]:
Y_test['Alpha_Predict'] = predicts[0]
Y_test['C_Predict'] = predicts[1]
Y_test['CA_Predict'] = predicts[2]
Y_test['N_Predict'] = predicts[3]
Y_test['MG_Predict'] = predicts[4]
Y_test['SI_Predict'] = predicts[5]
Y_test['NI_Predict'] = predicts[6]

In [96]:
Y_test

,teff_cnn,logg_cnn,feh_cnn,alpha_m_cnn,c_fe,ca_fe,n_fe,mg_fe,si_fe,ni_fe,Alpha_Predict,C_Predict,CA_Predict,N_Predict,MG_Predict,SI_Predict,NI_Predict
ID,,,,,,,,,,,,,,,,,
iDR4_3_SPLUS-n07s41_0026262,5165.86,4.476,-0.007,0.011,-0.05441,0.01777,-0.00575,0.01776,-0.03958,0.00848,0.013855,-0.006014,0.018461,-0.005952,0.053605,-0.008491,0.020344
iDR4_3_SPLUS-n02n26_0027037,5731.90,4.266,-0.153,0.003,0.01946,0.01651,-0.00447,0.01430,-0.00130,-0.00525,0.009073,0.008756,0.016074,0.089186,0.023908,0.008126,0.017829
iDR4_3_SPLUS-n02n27_0012652,5883.89,4.244,0.018,-0.013,0.00067,-0.00902,0.26882,-0.01945,-0.00811,0.05074,0.023268,0.016169,0.035428,0.099623,0.070154,0.027053,0.021105
iDR4_3_SPLUS-n01s38_0021247,5553.27,4.078,0.116,-0.015,0.01421,-0.01505,0.14115,-0.00363,0.00871,0.06575,-0.016024,-0.008967,-0.020240,0.194231,-0.019878,0.010909,0.039227
iDR4_3_SPLUS-n01s37_0039916,5281.96,4.407,0.057,-0.031,-0.01388,-0.01326,0.07569,-0.00113,-0.00467,0.02812,-0.027410,0.008069,-0.013871,0.068144,0.004190,0.011768,0.041480
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
iDR4_3_STRIPE82-0052_0030458,5767.40,4.324,-0.329,0.131,0.06415,0.11581,-0.04293,0.20228,0.11763,0.02593,0.111590,0.061908,0.119328,0.045814,0.136576,0.088102,0.021782
iDR4_3_SPLUS-n01s37_0032532,5079.59,4.307,-0.235,0.011,-0.08846,0.19101,-0.04492,-0.05606,-0.05711,-0.03312,0.010196,-0.012710,0.043561,0.056877,0.027418,0.008282,0.027860
iDR4_3_SPLUS-n02n27_0016721,5671.28,4.385,-0.045,-0.012,-0.02921,-0.01149,0.03796,-0.01192,-0.01303,-0.00211,0.004741,-0.009718,-0.012567,0.069444,0.071414,0.036365,0.028404


In [97]:
Y_test.to_csv(f"E:\\IC\\PredictsTreinos\\{NomeBase}\\{AouG}-OthersPredict.csv")

## Giants

In [ ]:
GigCores = pd.read_csv("E:\\IC\\BasesTreino\\LamostMedium-Colors-Giants.csv", index_col = 'ID')

In [ ]:
GigCores

In [ ]:
xtrain = GigCores.iloc[:, :66]

In [ ]:
columnYtrain =  ['teff_cnn', 'logg_cnn', 'feh_cnn', 'alpha_m_cnn', 
                 'c_fe', 'ca_fe', 'n_fe',
                 'mg_fe', 'si_fe', 'ni_fe']

In [ ]:
ytrain = GigCores[columnYtrain]

In [ ]:
ytrain['teffEncoded'] = encoder.encode_DataFrame(pd.DataFrame(ytrain['teff_cnn']), 'teffMedium')

In [ ]:
train_x= encoder.encode_DataFrame(xtrain, "NewCores") #The input vector: 66 stellar colors

train_y = ytrain

In [ ]:
# Separando a base para treino e para teste
X_train, X_test, Y_train, Y_test = train_test_split(train_x, train_y, test_size=0.2, random_state=11085)

In [ ]:
# Separando a base de treino para utilizar como validação no treinamento
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.1, random_state=11085)

In [ ]:
Y_train

### Teff | Logg | FEH

In [ ]:
Parametros = ["TEFF", "LOGG", "FEH"]
columnTrain = ['teffEncoded', 'logg_cnn', 'feh_cnn']

AouG = "Giants"
NomeBase = "LamostMedium"
BaseSaveModel = "Lamost-MR"

In [ ]:
batchSizes = [32, 64, 128, 256]
Epochs = [100, 200, 300, 400, 500]

In [ ]:
predicts = []
for i in range(0, 3, 1):
    predicts.append(train_model(Parametros[i], BaseSaveModel, AouG, batchSizes, Epochs, X_train, Y_train[columnTrain[i]], X_val, Y_val[columnTrain[i]], X_test, Y_test[columnTrain[i]]))